In [1]:
import pandas as pd
from plotly import graph_objects as go
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import optuna
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, mean_squared_error
import logging
from copy import deepcopy
from statsmodels.tsa.seasonal import seasonal_decompose
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools

d:\Programms\Programming\PythonProjects\A1_test\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data import

In [2]:
df = pd.read_excel('tasks_1-2.xlsx', sheet_name='Timeseries', date_format='%Y-%m-%d', parse_dates=['Date'])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     1642 non-null   datetime64[ns]
 1   series1  1642 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 25.8 KB


In [4]:
df.head(10)

,Date,series1
0,2015-01-01,1006.699649
1,2015-01-02,3197.751826
2,2015-01-03,3217.491035
3,2015-01-04,2151.573759
4,2015-01-05,4243.929892
5,2015-01-06,3178.012617
6,2015-01-07,1816.007210
7,2015-01-08,3020.098947
8,2015-01-09,3671.492837
9,2015-01-10,3138.534200


In [5]:
df['Date'].iloc[-1].date()

datetime.date(2019, 6, 30)

## Plots

In [6]:
# Plotting relation between date and series1 feature plot
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df['Date'], y=df['series1'], mode='lines')
)
fig.show()

In [7]:
# Seasonal decompose
decomposition_df = df.copy()
decomposition_df.set_index('Date', inplace=True)
decomposition = seasonal_decompose(decomposition_df, model='multiplicative')
fig = make_subplots(rows=4, cols=1,
                    subplot_titles=["Observed", "Trend", "Seasonal", "Residuals"],)
fig.add_trace(
    go.Scatter(x=decomposition_df.index, y=decomposition.observed, mode="lines", name='Observed'),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(x=decomposition_df.index, y=decomposition.trend, mode="lines", name='Trend'),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(x=decomposition_df.index, y=decomposition.seasonal, mode="lines", name='Seasonal'),
    row=3,
    col=1,
)
fig.add_trace(
    go.Scatter(x=decomposition_df.index, y=decomposition.resid, mode="lines", name='Residual'),
    row=4,
    col=1,
)
fig.update_layout(title='Seasonal decomposition', height=1080, width=1920)
fig.show()

## Time-series statistical tests

In [8]:
# ADF test
adf_test = adfuller(df['series1'])
print(f'ADF Statistic: {adf_test[0]}')
print(f'p-value: {adf_test[1]}')
print(f'Series is stationary (around unit root)? {"Yes" if adf_test[1] < 0.05 else "No"}')

ADF Statistic: -5.496713808722672
p-value: 2.1154012725975373e-06
Series is stationary (around unit root)? Yes


In [9]:
# KPSS test
kpss_test = kpss(df['series1'], regression='ct')
print(f'KPSS statistics: {kpss_test[0]}')
print(f'p-value: {kpss_test[1]}')
print(f'Series is stationary (around trend)? {"Yes" if kpss_test[1] > 0.05 else "No"}')

KPSS statistics: 0.1819907816436792
p-value: 0.022753456883620293
Series is stationary (around trend)? No


## Prophet + Optuna

In [10]:
# Bondary date for train/test split
boundary_date = df['Date'].iloc[-1].date() - relativedelta(months=3)
boundary_date

datetime.date(2019, 3, 30)

In [11]:
# Preparing datasets for Prophet
prophet_df_train = df[df['Date'].dt.date <= boundary_date].rename(columns={'Date': 'ds', 'series1': 'y'})
prophet_df_test = df[df['Date'].dt.date > boundary_date].rename(columns={'Date': 'ds', 'series1': 'y'})

In [12]:
def objective(trial: optuna.Trial):
    params = {
        'seasonality_prior_scale': trial.suggest_float('seasonality_prior_scale', 0.1, 100),
        'changepoint_prior_scale': trial.suggest_float('changepoint_prior_scale', 0.001, 10),
        'seasonality_mode': trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative']),
        'changepoint_range': trial.suggest_float('changepoint_range', 0.5, 1),
        'n_changepoints': trial.suggest_int('n_changepoints', 10, 200),                        
    }    
    ph = Prophet(weekly_seasonality=False,
                 daily_seasonality=False,
                 yearly_seasonality=True,                                               
                 **params)
    ph.add_seasonality(name='custom_seasonality',
                       period=trial.suggest_int('custom_seasonality_period', 30, 120),
                       fourier_order=trial.suggest_int('custom_seasonality_order', 2, 5))   
    ph.fit(prophet_df_train)
    # df_cv = cross_validation(ph, horizon='90 days', period='90 days', initial='730 days', parallel="processes")
    # metrics = performance_metrics(df_cv, rolling_window=1)
    df_future = ph.make_future_dataframe(periods=len(prophet_df_test), freq='D')
    df_predict = ph.predict(df_future)
    return root_mean_squared_error(df[df['Date'].dt.date > boundary_date]['series1'],
                                   df_predict[df_predict['ds'].dt.date > boundary_date]['yhat'])

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2024-11-19 11:43:56,509] A new study created in memory with name: no-name-aa5c86af-1ec5-441d-a174-c5018692c1ee
11:43:56 - cmdstanpy - INFO - Chain [1] start processing
11:43:59 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-11-19 11:43:59,239] Trial 0 finished with value: 1442.3679497520518 and parameters: {'seasonality_prior_scale': 4.070827647907315, 'changepoint_prior_scale': 7.6140362096304255, 'seasonality_mode': 'additive', 'changepoint_range': 0.8678713334527323, 'n_changepoints': 145, 'custom_seasonality_period': 52, 'custom_seasonality_order': 2}. Best is trial 0 with value: 1442.3679497520518.
11:43:59 - cmdstanpy - INFO - Chain [1] start processing
11:44:01 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-11-19 11:44:01,743] Trial 1 finished with value: 559.5935354387127 and parameters: {'seasonality_prior_scale': 20.125249651957734, 'changepoint_prior_scale': 3.4557913245962304, 'seasonality_mode': 'multiplicative', 'changepoint_range': 0.8632904873638112, 

In [13]:
# Initializing model with best params found by optuna
prophet_params = {k: study.best_params[k] for k in study.best_params
                  if k not in ['custom_seasonality_period', 'custom_seasonality_order']}
ph = Prophet(weekly_seasonality=False,
             daily_seasonality=False,
            #  yearly_seasonality=True,                          
             **prophet_params)
ph.add_seasonality(name='custom_seasonality',
                   period=study.best_params['custom_seasonality_period'],
                   fourier_order=study.best_params['custom_seasonality_order'])
ph.fit(prophet_df_train)
df_future = ph.make_future_dataframe(periods=len(prophet_df_test), freq='D')
df_future['ds'].iloc[-1].date()
df_predict = ph.predict(df_future)

11:46:31 - cmdstanpy - INFO - Chain [1] start processing
11:46:32 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
# Metrics after just Propher with some hyperparameters found by optuna
rmse_train = root_mean_squared_error(df[df['Date'].dt.date <= boundary_date]['series1'],
                                df_predict[df_predict['ds'].dt.date <= boundary_date]['yhat'])
rmse_test = root_mean_squared_error(df[df['Date'].dt.date > boundary_date]['series1'],
                               df_predict[df_predict['ds'].dt.date > boundary_date]['yhat'])
print(f'RMSE train: {rmse_train}')
print(f'RMSE test: {rmse_test}')

RMSE train: 794.4937004308996
RMSE test: 532.8146807733227


In [15]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df['Date'], y=df['series1'], mode='lines', name='Real data')    
)
fig.add_trace(
    go.Scatter(x=df_predict['ds'], y=df_predict['yhat'], mode='lines', name='Predicted data')
)
fig.add_trace(
    go.Scatter(x=df_predict['ds'], y=df_predict['yhat_upper'], mode='lines', name='Predicted data upper')
)
fig.add_trace(
    go.Scatter(x=df_predict['ds'], y=df_predict['yhat_lower'], mode='lines', name='Predicted data lower')
)
fig.update_layout(title='Prophet + Optuna')
fig.show()

In [16]:
# Three more month predict for Prophet + Optuna model
three_more_month_df = pd.DataFrame(data={'ds': pd.date_range(df['Date'].iloc[-1].date(),
                                                             df['Date'].iloc[-1].date() + relativedelta(months=3),
                                                             inclusive='right',
                                                             freq=timedelta(days=1),
                                                             normalize=True)})
three_more_month_predicted = ph.predict(three_more_month_df)
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df['Date'], y=df['series1'], mode='lines', name='Real data')    
)
fig.add_trace(
    go.Scatter(x=three_more_month_predicted['ds'],
               y=three_more_month_predicted['yhat'], mode='lines', name='3-month predicted data')
)
fig.add_trace(
    go.Scatter(x=three_more_month_predicted['ds'],
               y=three_more_month_predicted['yhat_upper'], mode='lines', name='3-month predicted data upper')
)
fig.add_trace(
    go.Scatter(x=three_more_month_predicted['ds'],
               y=three_more_month_predicted['yhat_lower'], mode='lines', name='3-month predicted data lower')
)
fig.update_layout(title='Prophet + Optuna (historical data + 3 new months prediction)')
fig.show()

In [17]:
# Only three more month predict for Prophet + Optuna model
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=three_more_month_predicted['ds'],
               y=three_more_month_predicted['yhat_lower'], mode='lines+markers',
               name='3-month predicted data lower')
)
fig.add_trace(
    go.Scatter(x=three_more_month_predicted['ds'],
               y=three_more_month_predicted['yhat'], mode='lines+markers',
               name='3-month predicted data',
               fill='tonexty')
)
fig.add_trace(
    go.Scatter(x=three_more_month_predicted['ds'],
               y=three_more_month_predicted['yhat_upper'], mode='lines+markers',
               name='3-month predicted data upper',
               fill='tonexty')
)


fig.update_layout(title='Prophet + Optuna (only 3 new months prediction)')
fig.show()

In [20]:
predicted_df = pd.DataFrame(columns=['Date', 'series1', 'series1_upper', 'series1_lower'],
                            data=three_more_month_predicted[['ds', 'yhat', 'yhat_upper', 'yhat_lower']].values)
predicted_df.to_csv('task_1_predictions.csv', index=False)